# Install `gofannon`

In [1]:
!pip install "git+https://github.com/The-AI-Alliance/gofannon.git@main#egg=gofannon[google]" --quiet

DEPRECATION: git+https://github.com/rawkintrevo/gofannon.git@31#egg=gofannon[google] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 67.8 MB/s eta 0:00:00


In [2]:
from openai import OpenAI
from google.colab import userdata
import json
from gofannon.google_search.google_search import GoogleSearch

google_search = GoogleSearch(api_key=userdata.get("google_search"), engine_id="75be790deec0c42f3")


# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=userdata.get('deepinfra'),
    base_url="https://api.deepinfra.com/v1/openai",
)

tool_list =  [google_search]
tool_map = {f.name: f.fn for f in tool_list}
tool_desc_list = [f.definition for f in tool_list]

In [5]:
messages = [
    {
        "role": "user",
        "content": "What newsworthy events happened at the Paris AI Summit?"
    }
]

# let's send the request and print the response
response = openai.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct",
    messages=messages,
    tools=tool_desc_list,
    tool_choice="auto",
)
tool_calls = response.choices[0].message.tool_calls
for tool_call in tool_calls:
    print(tool_call.model_dump())

{'id': 'call_nwPc50kSHkjiH5NV6wW6phWH', 'function': {'arguments': '{"query": "Paris AI Summit news", "num_results": 5}', 'name': 'google_search'}, 'type': 'function'}


In [6]:
messages.append(response.choices[0].message)

for tool_call in tool_calls:
  function_name = tool_call.function.name
  function_args = json.loads(tool_call.function.arguments)
  function_response = tool_map[function_name](**function_args)

  # extend conversation with function response
  messages.append({
      "tool_call_id": tool_call.id,
      "role": "tool",
      "content": function_response,
  })


# get a new response from the model where it can see the function responses
second_response = openai.chat.completions.create(
  model="meta-llama/Meta-Llama-3.1-70B-Instruct",
  messages=messages,
  tools=tool_desc_list,
  tool_choice="auto",
)

print(second_response.choices[0].message.content)

The Paris AI Summit, also known as the Artificial Intelligence Action Summit, was a recent event that brought together world leaders, CEOs, and experts to discuss the future of artificial intelligence. Some of the newsworthy events that took place at the summit include:

* France and the EU promising to cut red tape on tech regulation to make it easier for AI to flourish in the region
* Discussions on the destiny of the tech revolution and its potential impact on society
* Vice President JD Vance's address to European and Asian leaders, highlighting the importance of AI in shaping the future
* A call for projects from the Paris Peace Forum, aiming to address global challenges through AI
* A statement from Dario Amodei, appreciating the French government's efforts to bring together AI companies, researchers, and policymakers.

These events and discussions demonstrate the growing interest in AI and its potential to address global challenges, as well as the need for regulation and coopera

# Result

The Paris AI Summit took place on Feburary 10th and 11th 2025, well after Llama 3.3 was trained. Yet, by Googling the event, the LLM is able to provide an accurate recap of the event.